In [1]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

Using TensorFlow backend.


In [2]:
from keras.models import load_model
model = load_model('../models/facenet_vgg2.h5')

print(model.outputs)
print(model.inputs)











[<tf.Tensor 'Bottleneck_BatchNorm/batchnorm_1/add_1:0' shape=(?, 512) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]


/home/nathanalex/tf1/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 938 variables.
INFO:tensorflow:Converted 938 variables to const ops.


In [5]:
tf.train.write_graph(frozen_graph, "pb", "fn_vgg2.pb", as_text=False)

'pb/fn_vgg2.pb'